<a href="https://colab.research.google.com/github/Hanbin-git/dacon_new_drug/blob/main/20250710_%EC%84%B1%EB%8A%A5%ED%96%A5%EC%83%81%EC%9D%84%20%EC%9C%84%ED%95%B4%20meta%EB%AA%A8%EB%8D%B8%EC%A0%81%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip -o "/content/drive/MyDrive/data.zip" -d "/content/data_1"

Archive:  /content/drive/MyDrive/data.zip
  inflating: /content/data_1/sample_submission.csv  
  inflating: /content/data_1/test.csv  
  inflating: /content/data_1/train.csv  


In [3]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 63.8 MB/s eta 0:00:00


In [4]:
!pip install catboost xgboost lightgbm --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 22.4 MB/s eta 0:00:00


In [5]:
# import pandas as pd
# import os

# # ✅ Colab 경로
# extract_path = "/content/data_1"
# def get_path(filename):
#     return os.path.join(extract_path, filename)

# # ✅ 데이터 불러오기
# train = pd.read_csv(get_path("train.csv"))
# test = pd.read_csv(get_path("test.csv"))

# # ✅ 컬럼 확인
# print("✅ [train.csv] 컬럼 목록:")
# print(train.columns.tolist())

# print("\n✅ [test.csv] 컬럼 목록:")
# print(test.columns.tolist())

# # ✅ 샘플 데이터 미리 보기
# print("\n✅ [train.csv] 데이터 샘플:")
# print(train.head())

# print("\n✅ [test.csv] 데이터 샘플:")
# print(test.head())


✅ [train.csv] 컬럼 목록:
['ID', 'Canonical_Smiles', 'Inhibition']

✅ [test.csv] 컬럼 목록:
['ID', 'Canonical_Smiles']

✅ [train.csv] 데이터 샘플:
           ID                                   Canonical_Smiles  Inhibition
0  TRAIN_0000                        Cl.OC1(Cc2cccc(Br)c2)CCNCC1       12.50
1  TRAIN_0001                              Brc1ccc2OCCc3ccnc1c23        4.45
2  TRAIN_0002         CC1(CO)CC(=NO1)c2cc(c(F)cc2Cl)[N+](=O)[O-]        4.92
3  TRAIN_0003  Fc1ccc2nc(Nc3cccc(COc4cccc(c4)C(=O)N5CCOCC5)c3...       71.50
4  TRAIN_0004       CC(C)CC(=O)C1=C(Nc2c(Cl)ccc(Cl)c2C1=O)S(=O)C       18.30

✅ [test.csv] 데이터 샘플:
         ID                                   Canonical_Smiles
0  TEST_000  O=C(NC1CCCC1)C2(CCCCC2)N(CC3CCCO3)C(=O)C4=CNC(...
1  TEST_001                 CC(C)c1oc(Cn2ccnc2c3cncc(Br)c3)nc1
2  TEST_002  CC1=C(C(=O)NC(=O)N1)S(=O)(=O)NCC(Cc2ccccc2F)c3...
3  TEST_003     Cl.Cn1cc(cn1)c2cc(CN3CCc4nc[nH]c4C35CCOCC5)on2
4  TEST_004  CC(C)OC(=O)c1ccc(cc1)N2CC(CC2=O)C(=O)OCc3ccc(B...


In [6]:
# # ✅ import
# import pandas as pd
# import numpy as np
# import os
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Crippen, Lipinski, rdMolDescriptors, AllChem
# from sklearn.preprocessing import StandardScaler
# from catboost import CatBoostRegressor, Pool
# from lightgbm import LGBMRegressor
# from xgboost import XGBRegressor
# from sklearn.linear_model import Ridge
# from sklearn.model_selection import KFold

# # ✅ 경로 설정 (Colab 환경)
# extract_path = "/content/data_1"
# def get_path(filename):
#     return os.path.join(extract_path, filename)

# # ✅ 데이터 로딩
# train = pd.read_csv(get_path("train.csv"))
# test = pd.read_csv(get_path("test.csv"))
# submission = pd.read_csv(get_path("sample_submission.csv"))

# # ✅ RDKit Feature 추출
# def extract_rdkit_features(df):
#     mols = [Chem.MolFromSmiles(smi) for smi in df['Canonical_Smiles']]
#     features = {
#         'MolWt': [Descriptors.MolWt(mol) if mol else np.nan for mol in mols],
#         'LogP': [Crippen.MolLogP(mol) if mol else np.nan for mol in mols],
#         'NumHDonors': [Lipinski.NumHDonors(mol) if mol else np.nan for mol in mols],
#         'NumHAcceptors': [Lipinski.NumHAcceptors(mol) if mol else np.nan for mol in mols],
#         'TPSA': [rdMolDescriptors.CalcTPSA(mol) if mol else np.nan for mol in mols],
#         'NumRotatableBonds': [Descriptors.NumRotatableBonds(mol) if mol else np.nan for mol in mols],
#         'RingCount': [mol.GetRingInfo().NumRings() if mol else np.nan for mol in mols],
#         'HeavyAtomCount': [mol.GetNumHeavyAtoms() if mol else np.nan for mol in mols],
#         'FractionCSP3': [rdMolDescriptors.CalcFractionCSP3(mol) if mol else np.nan for mol in mols],
#         'NumAliphaticRings': [rdMolDescriptors.CalcNumAliphaticRings(mol) if mol else np.nan for mol in mols],
#         'NumAromaticRings': [rdMolDescriptors.CalcNumAromaticRings(mol) if mol else np.nan for mol in mols]
#     }
#     return pd.DataFrame(features)

# # ✅ Morgan Fingerprint 추출 (2048-bit)
# def get_morgan_fingerprint(smiles, radius=2, nBits=2048):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol:
#         fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
#         return np.array(fp)
#     else:
#         return np.zeros(nBits)

# def extract_morgan_df(df, nBits=2048):
#     fps = df['Canonical_Smiles'].apply(lambda x: get_morgan_fingerprint(x, nBits=nBits))
#     return pd.DataFrame(fps.tolist(), columns=[f'MFP_{i}' for i in range(nBits)])

# # ✅ Feature 생성
# X_train_rdkit = extract_rdkit_features(train)
# X_test_rdkit = extract_rdkit_features(test)
# X_train_morgan = extract_morgan_df(train)
# X_test_morgan = extract_morgan_df(test)

# # ✅ 결합
# X_train = pd.concat([X_train_rdkit, X_train_morgan], axis=1)
# X_test = pd.concat([X_test_rdkit, X_test_morgan], axis=1)
# y_train = train["Inhibition"]

# # ✅ 결측치 처리
# X_train.fillna(0, inplace=True)
# X_test.fillna(0, inplace=True)

# # ✅ 정규화
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
# X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# # ✅ Sample Weight
# sample_weight = np.log1p(y_train)

# # ✅ Stacking 앙상블
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# oof_cat = np.zeros(len(X_train))
# oof_lgb = np.zeros(len(X_train))
# oof_xgb = np.zeros(len(X_train))
# test_cat = np.zeros(len(X_test))
# test_lgb = np.zeros(len(X_test))
# test_xgb = np.zeros(len(X_test))

# for train_idx, val_idx in kf.split(X_train_df):
#     X_tr, X_val = X_train_df.iloc[train_idx], X_train_df.iloc[val_idx]
#     y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
#     w_tr = sample_weight.iloc[train_idx]

#     # CatBoost
#     cat = CatBoostRegressor(iterations=1500, learning_rate=0.03, depth=6,
#                             verbose=0, early_stopping_rounds=100, random_state=42)
#     cat.fit(Pool(X_tr, y_tr, weight=w_tr), eval_set=Pool(X_val, y_val))
#     oof_cat[val_idx] = cat.predict(X_val)
#     test_cat += cat.predict(X_test_df) / kf.n_splits

#     # LightGBM
#     lgb = LGBMRegressor(n_estimators=1500, learning_rate=0.03, max_depth=6, random_state=42)
#     lgb.fit(X_tr, y_tr, sample_weight=w_tr)
#     oof_lgb[val_idx] = lgb.predict(X_val)
#     test_lgb += lgb.predict(X_test_df) / kf.n_splits

#     # XGBoost
#     xgb = XGBRegressor(n_estimators=1500, learning_rate=0.03, max_depth=6, random_state=42)
#     xgb.fit(X_tr, y_tr, sample_weight=w_tr)
#     oof_xgb[val_idx] = xgb.predict(X_val)
#     test_xgb += xgb.predict(X_test_df) / kf.n_splits

# # ✅ 메타 모델
# stacked_train = np.vstack([oof_cat, oof_lgb, oof_xgb]).T
# stacked_test = np.vstack([test_cat, test_lgb, test_xgb]).T
# meta_model = Ridge(alpha=1.0)
# meta_model.fit(stacked_train, y_train)
# final_preds = meta_model.predict(stacked_test)

# # ✅ 제출 파일 저장
# submission['Inhibition'] = final_preds
# submission.to_csv('submission_stacking_rdkit_morgan.csv', index=False)
# print("✅ 'submission_stacking_rdkit_morgan.csv' 생성 완료!")


In [7]:
# import pandas as pd
# import numpy as np
# import os
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Crippen, Lipinski, rdMolDescriptors, AllChem
# from sklearn.preprocessing import StandardScaler
# from catboost import CatBoostRegressor, Pool

# # ✅ 데이터 로딩
# extract_path = "/content/data_1"  # Colab 기준
# train = pd.read_csv(os.path.join(extract_path, "train.csv"))
# test = pd.read_csv(os.path.join(extract_path, "test.csv"))
# submission = pd.read_csv(os.path.join(extract_path, "sample_submission.csv"))

# # ✅ RDKit 기반 화학 특성 추출 (13개 내외)
# def extract_physchem_features(df):
#     mols = [Chem.MolFromSmiles(smi) for smi in df['Canonical_Smiles']]
#     features = {
#         'MolWt': [Descriptors.MolWt(mol) if mol else np.nan for mol in mols],
#         'LogP': [Crippen.MolLogP(mol) if mol else np.nan for mol in mols],
#         'NumHDonors': [Lipinski.NumHDonors(mol) if mol else np.nan for mol in mols],
#         'NumHAcceptors': [Lipinski.NumHAcceptors(mol) if mol else np.nan for mol in mols],
#         'TPSA': [rdMolDescriptors.CalcTPSA(mol) if mol else np.nan for mol in mols],
#         'NumRotatableBonds': [Descriptors.NumRotatableBonds(mol) if mol else np.nan for mol in mols],
#         'RingCount': [mol.GetRingInfo().NumRings() if mol else np.nan for mol in mols],
#         'HeavyAtomCount': [mol.GetNumHeavyAtoms() if mol else np.nan for mol in mols],
#         'FractionCSP3': [rdMolDescriptors.CalcFractionCSP3(mol) if mol else np.nan for mol in mols],
#         'NumAliphaticRings': [rdMolDescriptors.CalcNumAliphaticRings(mol) if mol else np.nan for mol in mols],
#         'NumAromaticRings': [rdMolDescriptors.CalcNumAromaticRings(mol) if mol else np.nan for mol in mols],
#     }
#     return pd.DataFrame(features)

# # ✅ Morgan Fingerprint (2048 bit)
# def get_morgan_fp(smiles, radius=2, nBits=2048):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol:
#         return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits))
#     else:
#         return np.zeros(nBits)

# def extract_morgan_df(df, nBits=2048):
#     fps = df['Canonical_Smiles'].apply(lambda x: get_morgan_fp(x, nBits=nBits))
#     return pd.DataFrame(fps.tolist(), columns=[f'MFP_{i}' for i in range(nBits)])

# # ✅ 피처 생성
# X_train_phys = extract_physchem_features(train)
# X_test_phys = extract_physchem_features(test)
# X_train_fp = extract_morgan_df(train)
# X_test_fp = extract_morgan_df(test)

# # ✅ 피처 결합
# X_train = pd.concat([X_train_phys, X_train_fp], axis=1)
# X_test = pd.concat([X_test_phys, X_test_fp], axis=1)
# y_train = train["Inhibition"]

# # ✅ 정규화
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # ✅ Sample Weighting (log scale)
# sample_weight = np.log1p(y_train)

# # ✅ CatBoost 학습
# model = CatBoostRegressor(
#     iterations=1500,
#     learning_rate=0.03,
#     depth=6,
#     loss_function='RMSE',
#     early_stopping_rounds=100,
#     verbose=100,
#     random_state=42
# )

# model.fit(Pool(X_train_scaled, y_train, weight=sample_weight))
# preds = model.predict(X_test_scaled)

# # ✅ 제출 파일 저장
# submission["Inhibition"] = preds
# submission.to_csv("submission_catboost_morgan_physchem13.csv", index=False)
# print("✅ 'submission_catboost_morgan_physchem13.csv' 저장 완료")


In [8]:
# # ✅ 라이브러리
# import os
# import numpy as np
# import pandas as pd
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Crippen, Lipinski, rdMolDescriptors, AllChem
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import Ridge
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# from catboost import CatBoostRegressor, Pool
# from lightgbm import LGBMRegressor
# from xgboost import XGBRegressor

# # ✅ 경로 설정 (Colab 기준)
# extract_path = "/content/data_1"
# def get_path(filename): return os.path.join(extract_path, filename)

# train = pd.read_csv(get_path("train.csv"))
# test = pd.read_csv(get_path("test.csv"))
# submission = pd.read_csv(get_path("sample_submission.csv"))

# # ✅ RDKit 기반 분자 특성 추출
# def extract_rdkit_features(df):
#     mols = [Chem.MolFromSmiles(smi) for smi in df['Canonical_Smiles']]
#     features = {
#         'MolWt': [Descriptors.MolWt(mol) if mol else np.nan for mol in mols],
#         'LogP': [Crippen.MolLogP(mol) if mol else np.nan for mol in mols],
#         'NumHDonors': [Lipinski.NumHDonors(mol) if mol else np.nan for mol in mols],
#         'NumHAcceptors': [Lipinski.NumHAcceptors(mol) if mol else np.nan for mol in mols],
#         'TPSA': [rdMolDescriptors.CalcTPSA(mol) if mol else np.nan for mol in mols],
#         'NumRotatableBonds': [Descriptors.NumRotatableBonds(mol) if mol else np.nan for mol in mols],
#         'RingCount': [mol.GetRingInfo().NumRings() if mol else np.nan for mol in mols],
#         'HeavyAtomCount': [mol.GetNumHeavyAtoms() if mol else np.nan for mol in mols],
#         'FractionCSP3': [rdMolDescriptors.CalcFractionCSP3(mol) if mol else np.nan for mol in mols],
#         'NumAliphaticRings': [rdMolDescriptors.CalcNumAliphaticRings(mol) if mol else np.nan for mol in mols],
#         'NumAromaticRings': [rdMolDescriptors.CalcNumAromaticRings(mol) if mol else np.nan for mol in mols]
#     }
#     return pd.DataFrame(features)

# # ✅ Morgan Fingerprint
# def get_morgan_fingerprint(smiles, radius=2, nBits=2048):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol:
#         fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits)
#         return np.array(fp)
#     else:
#         return np.zeros(nBits)

# def extract_morgan_df(df, nBits=2048):
#     fps = df['Canonical_Smiles'].apply(lambda x: get_morgan_fingerprint(x, nBits=nBits))
#     return pd.DataFrame(fps.tolist(), columns=[f'MFP_{i}' for i in range(nBits)])

# # ✅ Feature 결합
# X_train_rdkit = extract_rdkit_features(train)
# X_test_rdkit = extract_rdkit_features(test)
# X_train_morgan = extract_morgan_df(train)
# X_test_morgan = extract_morgan_df(test)

# X_train = pd.concat([X_train_rdkit, X_train_morgan], axis=1)
# X_test = pd.concat([X_test_rdkit, X_test_morgan], axis=1)
# y_train = train["Inhibition"]

# # ✅ 정규화
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
# X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# # ✅ Sample Weight
# sample_weight = np.log1p(y_train)

# # ✅ 앙상블 초기화
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# oof_cat = np.zeros(len(X_train))
# oof_lgb = np.zeros(len(X_train))
# oof_xgb = np.zeros(len(X_train))
# test_cat = np.zeros(len(X_test))
# test_lgb = np.zeros(len(X_test))
# test_xgb = np.zeros(len(X_test))

# # ✅ 교차검증 모델 학습
# for train_idx, val_idx in kf.split(X_train_df):
#     X_tr, X_val = X_train_df.iloc[train_idx], X_train_df.iloc[val_idx]
#     y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
#     w_tr = sample_weight.iloc[train_idx]

#     # ✅ CatBoost (GPU)
#     cat = CatBoostRegressor(
#         iterations=1500, learning_rate=0.03, depth=6, verbose=0,
#         early_stopping_rounds=100, random_state=42, task_type='GPU'
#     )
#     cat.fit(Pool(X_tr, y_tr, weight=w_tr), eval_set=Pool(X_val, y_val))
#     oof_cat[val_idx] = cat.predict(X_val)
#     test_cat += cat.predict(X_test_df) / kf.n_splits

#     # ✅ LightGBM (GPU)
#     lgb = LGBMRegressor(
#         n_estimators=1500, learning_rate=0.03, max_depth=6, random_state=42,
#         device='gpu'
#     )
#     lgb.fit(X_tr, y_tr, sample_weight=w_tr)
#     oof_lgb[val_idx] = lgb.predict(X_val)
#     test_lgb += lgb.predict(X_test_df) / kf.n_splits

#     # ✅ XGBoost (GPU)
#     xgb = XGBRegressor(
#         n_estimators=1500, learning_rate=0.03, max_depth=6, random_state=42,
#         tree_method='gpu_hist', predictor='gpu_predictor'
#     )
#     xgb.fit(X_tr, y_tr, sample_weight=w_tr)
#     oof_xgb[val_idx] = xgb.predict(X_val)
#     test_xgb += xgb.predict(X_test_df) / kf.n_splits

# # ✅ 메타 모델 (Ridge)
# stacked_train = np.vstack([oof_cat, oof_lgb, oof_xgb]).T
# stacked_test = np.vstack([test_cat, test_lgb, test_xgb]).T
# meta_model = Ridge(alpha=1.0)
# meta_model.fit(stacked_train, y_train)
# final_preds = meta_model.predict(stacked_test)

# # ✅ 성능 평가
# rmse = np.sqrt(mean_squared_error(y_train, meta_model.predict(stacked_train)))
# mae = mean_absolute_error(y_train, meta_model.predict(stacked_train))
# print(f"✅ Meta Model RMSE: {rmse:.4f}")
# print(f"✅ Meta Model MAE : {mae:.4f}")

# # ✅ 제출
# submission['Inhibition'] = final_preds
# submission.to_csv("submission_stacking_rdkit_morgan.csv", index=False)
# print("✅ 'submission_stacking_rdkit_morgan.csv' 저장 완료!")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

...

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2776
[LightGBM] [Info] Number of data points in the train set: 1345, number of used features: 608
[LightGBM] [Info] Using GPU Device: NVIDIA A100-SXM4-40GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 26 dense feature groups (0.04 MB) transferred to GPU in 0.001691 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 43.249144
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:15:33] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:15:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:15:41] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2764
[LightGBM] [Info] Number of data points in the train set: 1345, number of used features: 604
[LightGBM] [Info] Using GPU Device: NVIDIA A100-SXM4-40GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 26 dense feature groups (0.04 MB) transferred to GPU in 0.001547 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 42.427172
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:15:48] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:15:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:15:57] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


✅ Meta Model RMSE: 23.8237
✅ Meta Model MAE : 19.6814
✅ 'submission_stacking_rdkit_morgan.csv' 저장 완료!
